---

## Задание 1: Теоретическое описание задачи

Задача извлечения именованных сущностей (Named Entity Recognition, NER) — это ключевая подзадача обработки естественного языка (NLP), направленная на автоматическое обнаружение и классификацию сущностей в неструктурированном тексте. К типичным категориям сущностей относятся:

- `PER` — персоны (люди),
- `ORG` — организации,
- `LOC` — географические объекты,
- `EVT` — события,
- `PRO` — продукты или объекты.

**Классические методы:**
До распространения нейронных сетей основными подходами были:

- Правила и шаблоны (rule-based systems),
- Статистические модели: HMM (Hidden Markov Models), CRF (Conditional Random Fields),
- Feature engineering — ручное составление признаков на основе лингвистических свойств.

**Современные подходы (LLM):** 

С появлением больших языковых моделей (LLM) стало возможным извлечение сущностей в zero-shot или few-shot режимах, без дополнительного обучения. GigaChat, как представитель LLM, способен воспринимать структурированный prompt и возвращать списки сущностей, опираясь на контекст.

**Метрики качества:**

Для оценки качества предсказания применяются:

Precision (точность),
Recall (полнота),
F1-score — гармоническое среднее между precision и recall.
Оценка может проводиться:

По каждой сущности (class-wise),
В среднем (macro/micro-average).

---

##  Задание 2: Загрузка и подготовка датасета

**Формулировка:**  
Реализуйте чтение датасета в pandas DataFrame с обязательными колонками:
- `document_id`
- `document_text`
- `entity`
- `gold_answer`

Выведите шапку датафрейма.


In [15]:
import sys, os
import pandas as pd


sys.path.insert(0, os.path.abspath('..'))

def load_annotations(annotation_dir: str) -> pd.DataFrame:
    records = []
    for fname in os.listdir(annotation_dir):
        if not fname.endswith('.out'):
            continue
        doc_id = os.path.splitext(fname)[0]
        with open(os.path.join(annotation_dir, fname), encoding='utf-8') as f:
            lines = [L.strip() for L in f if L.strip()]
        if lines and lines[0] == doc_id.split('_')[-1]:
            lines = lines[1:]
        for line in lines:
            parts = line.rsplit(None, 3)
            if len(parts) >= 4:
                surface, lemma, ent_type, canonical = parts
                records.append({
                    'document_id': doc_id,
                    'entity': ent_type,
                    'gold_answer': surface
                })
    return pd.DataFrame(records)

def load_texts(text_dir: str, skip_lines: int = 4) -> pd.DataFrame:
    records = []
    for fname in os.listdir(text_dir):
        if not fname.endswith('.txt'):
            continue
        doc_id = os.path.splitext(fname)[0]
        with open(os.path.join(text_dir, fname), encoding='utf-8') as f:
            lines = f.read().splitlines()
        body = "\n".join(lines[skip_lines:]).strip()
        records.append({
            'document_id': doc_id,
            'document_text': body
        })
    return pd.DataFrame(records)




df.head()

from src.data_loader import load_annotations, load_texts



ANN_DIR = '../data/annotations'
TXT_DIR = '../data/raw'

df_ann = load_annotations(ANN_DIR)
df_txt = load_texts   (TXT_DIR)
df     = df_ann.merge(df_txt, on='document_id', how='left')
display(df.head(100))





,document_id,entity,gold_answer,document_text
0,brexit_ru.txt_file_10,EVT,Brexit,Тереза Мэй рассчитывает усидеть в седле до зав...
1,brexit_ru.txt_file_10,LOC,Альбиона,Тереза Мэй рассчитывает усидеть в седле до зав...
2,brexit_ru.txt_file_10,LOC,Альбионе,Тереза Мэй рассчитывает усидеть в седле до зав...
3,brexit_ru.txt_file_10,PER,Борис Джонсон Борис,Тереза Мэй рассчитывает усидеть в седле до зав...
4,brexit_ru.txt_file_10,LOC,Британии,Тереза Мэй рассчитывает усидеть в седле до зав...
...,...,...,...,...
95,brexit_ru.txt_file_1004,PER,Терезы,Борис Джонсон ушел в отставку с поста главы МИ...
96,brexit_ru.txt_file_1006,EVT,Brexit,Захарова лирически прокомментировала отставку ...
97,brexit_ru.txt_file_1006,PRO,Facebook,Захарова лирически прокомментировала отставку ...
98,brexit_ru.txt_file_1006,PER,Борис Джонсон Борис,Захарова лирически прокомментировала отставку ...


---

##  Задание 3: Формирование prompt-функции для LLM

**Формулировка:**  
Напишите функцию, которая принимает на вход строку датафрейма и выдает текст входного сообщения для LLM




In [ ]:
# Код для задания 3

---

##  Задание 4: Ручное получение ответов от GigaChat и сохранение

**Формулировка:**  
Получите ответы GigaChat для всех документов. Документов всего 9, поэтому сделать это можно вручную, пользуясь веб-интерфейсом GigaChat или ботом в ВК или Телеграме. 




In [ ]:
# Код для задания 4

---

##  Задание 5: Реализация метрики score_fn() и юнит-тесты

**Формулировка:**  
<сюда можно вставить формулировку из ТЗ>




In [ ]:
# Код для задания 5

---

##  Задание 6: Расчёт метрик и визуализация по сущностям/документам

**Формулировка:**  
<сюда можно вставить формулировку из ТЗ>



In [ ]:
# Код для задания 6


---

##  Задание 7: Анализ зависимости качества от длины документа

**Формулировка:**  
<сюда можно вставить формулировку из ТЗ>


In [ ]:
# Код для задания 7

---

##  Задание 8: Анализ ошибок, слабых и сильных сторон модели

**Формулировка:**  




In [ ]:
# Код для задания 8

---

##  Задание 9: Выводы по проекту и самооценка

**Формулировка:**  





In [ ]:
# Код для задания 9